数据来源平台：爬取印尼四大电商平台数据：lazada，shopee，tokopedia，bukalapak

爬取品类 ：按顺序依次为：鞋，表，包，饰品

价位区间：按200-300，及300以上两个价位区间爬取(饰品稍等，待观察各品台基础价位)

销量：爬出月销量1500以上的sku

https://www.tokopedia.com/

https://www.bukalapak.com/


注意价格中的逗号
price_range = re.sub(r'\,|\n','',price_range)

## shopee
月销
## 奇怪 Get 不到 product name

In [7]:
from selenium import webdriver
import time,re,csv
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
import selenium.webdriver.support.ui as WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys

def get_list(class_name,class_url,save_file,monthly_sales):
    
    # Chrome object

    print('Getting class item list from Shopee ...')
    driver = webdriver.Chrome()  
    driver.maximize_window()
    driver.get(class_url) 
    time.sleep(2)

    for i in range(4):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        

    items = driver.find_elements_by_css_selector('div.col-xs-2-4.shopee-search-item-result__item')
    item_number = len(items)
    if item_number == 50:
        with open(save_file,'a',encoding='utf-8') as w:    
            for i in items:
                #print(i.find_element_by_css_selector('._1NoI8_._2gr36I'))
                
                #pro_name = i.find_element_by_css_selector('div._1NoI8_._2gr36I').text
                pro_name = i.find_element_by_css_selector("[class^=_1NoI8_]").text
                price_info  = i.find_element_by_css_selector('div._36zw98').text
                
                price_range = re.sub(r'\.','',price_info)
    
                if '-' in price_range:
                    a = re.findall('Rp(.*) - Rp(.*)',price_range)
                    price = (int(a[0][0])+int(a[0][1]))/2
                else:
                    price = int(re.search('Rp(.*)',price_range).group(1))
                
                info = i.find_element_by_css_selector('div._2-i6yP').text
                monthly_sales = int(re.search('(\d+) Terjual',info).group(1))
                pro_url = i.find_element_by_tag_name('a').get_attribute('href')
                pic_info = i.find_element_by_css_selector('div._1T9dHf._3XaILN').get_attribute('style')
                pic_url = re.search(r'url\("(.*)"\);',pic_info).group(1)
                if monthly_sales >= sales_select:
                    print('class_name:{}\nproduct name:{}\nprice_info:{}\nprice:{}\nproduct url:{}\nmonthly sales:{}\npicture url:{}\n'.format(class_name,pro_name,price_info,price,pro_url,monthly_sales,pic_url))
                    #w.write('{},{}\n'.foramt(url,price))
                
    else:
        print('number of items is{}, waiting 10 seconds to reload'.foramt(item_number))
        #time.sleep(10)
        #get_list(class_url,save_file)

    
class_name = '靴子'
class_url = 'https://shopee.co.id/Boots-cat.35.1033?page=0&sortBy=sales'
save_file = r'class_item_list_over3000.csv'
sales_select = 1500

get_list(class_name,class_url,save_file,sales_select)

Getting class item list from Shopee ...
class_name:靴子
product name:
price_info:Rp198.000 - Rp279.000
price:238500.0
product url:https://shopee.co.id/Sepatu-Boots-Humm3r-Golem-size-39-44-dan-45-48-i.40377301.933066496
monthly sales:1577
picture url:https://cf.shopee.co.id/file/c11757c079d3611d2ff9e2beef2ef41a_tn



## pyquery

### 奇怪 get 不到 product url
`pro_url = i.find('div a').attr('href') `

#### 原因是打印出html代码，会发现包含  `xmlns="http://www.w3.org/1999/xhtml"`

```
<div xmlns="http://www.w3.org/1999/xhtml" class="col-xs-2-4 shopee-search-item-result__item"><div><a href="/Sepatu-Boots-Humm3r-Golem-size-39-44-dan-45-48-i.40377301.933066496"><div class="_1gkBDw"><div class="_3ZDC1p"><div class="_1T9dHf _3XaILN" style="background-image: url(&quot;https://cf.shopee.co.id/file/c11757c079d3611d2ff9e2beef2ef41a_tn&quot;)
```

但是pyquery中的选择器并没有错误，但是运行结果一直是None。这是为什么呢？后来通过查看相关文档得知，pyquery解析的是html类型的字符串，但是上面的类型是xhtml，所以会获取不到元素。可以在pq()方法初始化字符串时加上parser="html"告诉pyquery使用html规范解析，即可解决上述问题。
https://blog.csdn.net/qq_40176258/article/details/85041089

`pq(html,parser="html")`


再次打印出html代码，会发现已经不包含  `xmlns="http://www.w3.org/1999/xhtml"`

[官方文档](https://pythonhosted.org/pyquery/tips.html?highlight=xhtml)

In [ ]:
from selenium import webdriver
import time,re,csv,pymongo,os
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys


def get_list(driver,save_file,category,class_name,class_url,url_format,sales_select,page):
    
    print('page {} ...'.format(page)) 
    driver.get(class_url.format(url_format,page)) 
    time.sleep(2)
    for i in range(5):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(4)
    pic_num = len(driver.find_elements_by_css_selector('._1w9jLI._37ge-4._2XtIUk'))
    print('picture load {}/50...'.format(pic_num))
    while (pic_num<49):
        driver.refresh()
        for i in range(4):
            driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
        for i in range(4):
            driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_UP)
            time.sleep(2)
        pic_num = len(driver.find_elements_by_css_selector('._1w9jLI._37ge-4._2XtIUk'))

    html = driver.page_source
    doc = pq(html,parser="html")
    item_list = doc('.col-xs-2-4.shopee-search-item-result__item')
    num = 0
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        for i in item_list.items():
            try:
                info = i.find('._2-i6yP').text()
                print(info)
                monthly_sales_info = re.search('(\d+) Terjual',info)
                if monthly_sales_info:
                    monthly_sales = int(monthly_sales_info.group(1))
                if monthly_sales >= sales_select:
                    num += 1
                    pro_name = i.find('._1NoI8_._2gr36I').text()
                    pro_name = re.sub(r',',' ',pro_name)
                    price_info  = i.find('._1w9jLI._37ge-4._2XtIUk').text()
                    price_range = re.sub(r'\.|\n','',price_info)
                    if '-' in price_range:
                        a = re.findall('Rp(.*) - Rp(.*)',price_range)
                        price = (int(a[0][0])+int(a[0][1]))/2
                    else:
                        price = int(re.search('Rp(.*)',price_range).group(1))
                    prefix = 'https://shopee.co.id'
                    pro_url = prefix+i.find('div a').attr('href') 
                    pic_info = i.find('._1T9dHf._3XaILN').attr('style')
                    print(type(pic_info))
                    pic_url = re.search(r'url\("(.*)_tn"\);',str(pic_info)).group(1).strip()
                    print('category:{}\nclass name:{}\nproduct name:{}\nprice info:{}\nprice:{}\nproduct url: {}\nmonthly sales:{}\npicture url: {}\n'.format(category,class_name,pro_name,price_range,price,pro_url,monthly_sales,pic_url))
                    csvfile.write('{},{},{},{},{},{},{},{}\n'.format(category,class_name,pro_name,price_range,price,monthly_sales,pro_url,pic_url)) 
                    pic_name = ' '.join(re.sub('[^a-z0-9A-Z_\-\s]','',pro_name,count=0).split())
                    save_image(pic_url,pic_name,category)

                    item = {
                        'category':category,
                        'main class':class_name,
                        'item name':pro_name,
                        'price info':price_range,
                        'price':price,
                        'monthly sales':monthly_sales,
                        'item url':pro_url,
                        'item picture':pic_url
                    }
                    save_to_mongo(item)
            except:
                print('something got worng, pass this item {}'.format(pro_name))
                continue
    print(num,len(item_list))
    if num == len(item_list):
        page += 1
        get_list(driver,save_file,category,class_name,class_url,url_format,sales_select,page)
                
        
def save_image(img_url,pic_name,category):
    print('{} saving picture... {}'.format(pic_name,img_url))
    img_response = requests.get(img_url,headers=headers)
    path = r'{}'.format(category)
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
    with open(r'{}\{}.png'.format(path,pic_name),'ab') as f:
        f.write(img_response.content)          
                        
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'sales_crawl'
    MONGO_COLLECTION = 'shopee_sales_1500_0305'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


def get_url(file,driver,save_file,sales_select,page): 
    
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['category','main class','item name','price info','price','monthly sales','item url','item picture']) 
    with open(file,'r',encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = [row for row in reader][-3:]
        for row in rows:
            category = row[0]
            class_name = row[1]
            url_format = row[2]
            get_list(driver,save_file,category,class_name,class_url,url_format,sales_select,page)
            
file = 'sales.csv'  
headers = {'User-Agent': 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}            
#category = 'MenShoes'
#class_name = 'Boots'
#class_url = 'https://shopee.co.id/Boots-cat.35.1033?page={}&sortBy=sales'

class_url = 'https://shopee.co.id/{}?page={}&sortBy=sales'
save_file = r'shopee_sales_1500_0305.csv'
sales_select = 1500
page = 0

if __name__ == '__main__':
    driver = webdriver.Chrome()
    driver.maximize_window()
    get_url(file,driver,save_file,sales_select,page)
    driver.close()

In [6]:
import csv

def get_url(file):
    
    with open(file,'r',encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = [row for row in reader][1:]
        for row in rows:
            print(row[0],row[1],row[2])
            
file = 'sales.csv'
get_url(file)

MenShoes Boots Boots-cat.35.1033
MenShoes CasualShoes Sepatu-Kasual-cat.35.1027
MenShoes Sandals Sandal-cat.35.1019
MenShoes Sneakers Sneakers-cat.35.1013
MenShoes Formal Sepatu-Formal-cat.35.1031
Watches MenWatches Jam-Tangan-Pria-cat.37.12452
Watches WomenWatches Jam-Tangan-Wanita-cat.37.12443
Watches CoupleWatches Jam-Tangan-Couple-cat.37.12461
Menbags Backpacks Tas-Punggung-cat.10217.10222
Menbags ToteBags Tas-Tote-cat.10217.12621
Menbags MessengeBags Tas-Messenger-cat.10217.18222
Menbags SlingBags Tas-Selempang-cat.10217.18224
Menbags WaistBags Tas-Pinggang-cat.10217.12631
Menbags LaptopBags Tas-Laptop-cat.10217.10223
Menbags Shoulder Bags Tas-Bahu-cat.10217.10224
Menbags ClutchBags Clutch-cat.10217.18227
Menbags SportsBags Tas-Olahraga-cat.10217.12626
Menbags MenWallet Dompet-Pria-cat.10217.10219
Fashion HairAccessories Aksesoris-Rambut-cat.38.1145
Fashion Hats Topi-cat.38.1147
Fashion Glasses Kacamata-cat.38.1143
Fashion ContactLenses Lensa-Kontak-cat.38.14983
Fashion Earrings A

### 报错 TypeError: expected string or bytes-like object
（预定的数据类型或者字节对象相关） 一般为数据类型不匹配造成的

```
---> 49                 pic_url = re.search(r'url\("(.*)_tn"\);',pic_info).group(1).strip()
     50                 print('category:{}\nclass name:{}\nproduct name:{}\nprice info:{}\nprice:{}\nproduct url: {}\nmonthly sales:{}\npicture url: {}\n'.format(category,class_name,pro_name,price_range,price,pro_url,monthly_sales,pic_url))
     51                 csvfile.write('{},{},{},{},{},{},{},{}\n'.format(category,class_name,pro_name,price_range,price,monthly_sales,pro_url,pic_url))

c:\users\win7\appdata\local\programs\python\python36-32\lib\re.py in search(pattern, string, flags)
    180     """Scan through string looking for a match to the pattern, returning
    181     a match object, or None if no match was found."""
--> 182     return _compile(pattern, flags).search(string)
    183 
    184 def sub(pattern, repl, string, count=0, flags=0):

TypeError: expected string or bytes-like object
```
### 原因是 
`pic_info = i.find('._1T9dHf._3XaILN').attr('style')` 返回的是 `nonetype` 类型


用正则表达式处理数据时时出现错误,正则是用来匹配字符串类型，
因此在正则表达式之前，转换一下数据类型，就解决了问题
str()

[参考文档](https://blog.csdn.net/weixin_42105977/article/details/80390957)

## Shopee TW

麻烦在虾皮台湾官网，爬取如下价格品类的爆款数据：
 
    https://shopee.tw/美妝保健-cat.67?page=0&sortBy=sales
    https://shopee.tw/居家生活-cat.73
    https://shopee.tw/女生配件-cat.2580
    https://shopee.tw/寵物-cat.1657
    https://shopee.tw/女鞋-cat.64
    https://shopee.tw/男鞋-cat.1859
    https://shopee.tw/女生包包-精品-cat.65
    https://shopee.tw/男生包包與配件-cat.1837

取价格在台币800以上的产品，输入字段：品类，产品名称，URL，价格，月销量
月销量 500 以上

In [1]:
from selenium import webdriver
import time,re,csv,pymongo,os
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys


def get_list(driver,save_file,category,class_name,class_url,url_format,price_select,sales_select,page):
    
    target_url = class_url.format(url_format,page)
    print('class url: {} , page: {} ...'.format(target_url,page))
    driver.get(target_url) 
    time.sleep(2)
    for i in range(5):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(4)
    pic_num = len(driver.find_elements_by_css_selector('._1T9dHf._3XaILN'))
    print('picture load {}/50...'.format(pic_num))
    while (pic_num<49):
        driver.refresh()
        for i in range(4):
            driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
        for i in range(4):
            driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_UP)
            time.sleep(2)
        pic_num = len(driver.find_elements_by_css_selector('._1T9dHf._3XaILN'))
        print('picture load again {}/50...'.format(pic_num))

    html = driver.page_source
    doc = pq(html,parser="html")
    item_list = doc('.col-xs-2-4.shopee-search-item-result__item')
    num = 0
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for i in item_list.items():
            try:
                sales_info = i.find('._2-i6yP').text()
                
                monthly_sales_info = re.search('月銷量 (\d+)',sales_info)
                if monthly_sales_info:
                    monthly_sales = int(monthly_sales_info.group(1))
                else:
                    monthly_sales = 0
                if monthly_sales > sales_select:
                    num += 1
                    price_info  = i.find('._1w9jLI._37ge-4._2XtIUk').text()
                    price_range = re.sub(r'\,|\n','',price_info)
                    #print(price_range)
                    if '-' in price_range:
                        a = re.findall('\$(\d+) - \$(\d+)',price_range)
                        price_min = int(a[0][0])
                        price_max = int(a[0][1])
                    else:
                        price = int(re.search('\$(\d+)',price_range).group(1))
                        price_min = price
                        price_max = price
                    if price_max > price_select:
                        pro_name = i.find('._1NoI8_._2gr36I').text()
                        pro_name = re.sub(r',',' ',pro_name)
                        sales_info = i.find('._2-i6yP').text()
                        #print(sales_info)
                        prefix = 'https://shopee.tw'
                        pro_url = prefix+i.find('div a').attr('href') 
                        pic_info = i.find('._1T9dHf._3XaILN').attr('style')
                        #print(type(pic_info))
                        pic_url = re.search(r'url\("(.*)_tn"\);',str(pic_info)).group(1).strip()
                        #print(pic_url)
                        print('category:{}\nclass name:{}\nproduct name:{}\nprice info:{}\nprice_min:{}\nprice_max:{}\nproduct url: {}\nmonthly sales:{}\npicture url: {}\n'.format(category,class_name,pro_name,price_range,price_min,price_max,pro_url,monthly_sales,pic_url))
                        csvfile.write('{},{},{},{},{},{},{},{},{}\n'.format(category,class_name,pro_name,price_range,price_min,price_max,monthly_sales,pro_url,pic_url)) 
                        pic_name = ' '.join(re.sub('[^a-z0-9A-Z_\-\s]','',pro_name,count=0).split())
                        #save_image(pic_url,pic_name,category)
                        item = {
                            'category':category,
                            'main class':class_name,
                            'item name':pro_name,
                            'price info':price_range,
                            'price min':price_min,
                            'price max':price_max,
                            'monthly sales':monthly_sales,
                            'item url':pro_url,
                            'item picture':pic_url
                        }
                        save_to_mongo(item)
            except Exception as e:
                print('something got worng {}, pass this item {}'.format(e,pro_name))
                continue       
    #print(num,len(item_list))
    if num == len(item_list):
        page += 1
        get_list(driver,save_file,category,class_name,class_url,url_format,price_select,sales_select,page)
        
def save_image(img_url,pic_name,category):
    print('{} saving picture... {}'.format(pic_name,img_url))
    img_response = requests.get(img_url,headers=headers)
    path = r'{}'.format(category)
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
    with open(r'{}\{}.png'.format(path,pic_name),'ab') as f:
        f.write(img_response.content)          

def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopee_crawl'
    MONGO_COLLECTION = 'shopee_tw_no_limit_price'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


def get_url(file,driver,save_file,price_select,sales_select,page): 
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['category','main class','item name','price range','price_min','price_max','monthly sales','item url','item picture']) 
    with open(file,'r',encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = [row for row in reader]
        for row in rows[1:]:
            category = row[0]
            class_name = row[1]
            url_format = row[2]
            get_list(driver,save_file,category,class_name,class_url,url_format,price_select,sales_select,page)
            
file = 'shopee_tw.csv'  
headers = {'User-Agent': 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}            


class_url = 'https://shopee.tw/{}?page={}&sortBy=sales'
save_file = r'shopee_tw_no_limit_price.csv'
price_select = 0
sales_select = 500
page = 0

if __name__ == '__main__':
    driver = webdriver.Chrome()
    driver.maximize_window()    
    get_url(file,driver,save_file,price_select,sales_select,page)
    driver.close()

class url: https://shopee.tw/開架流行清潔保養-cat.67.15401?page=0&sortBy=sales , page: 0 ...
picture load 50/50...
category:美妝保健
class name:開架流行清潔保養
product name:【明洞妞】💥全館699免運💥韓國Wonjin Effect 原辰保濕面膜 30g 保濕面膜 吊瓶面膜 植物細胞
price info:$25
price_min:25
price_max:25
product url: https://shopee.tw/【明洞妞】💥全館699免運💥韓國Wonjin-Effect-原辰保濕面膜-30g-保濕面膜-吊瓶面膜-植物細胞-i.1944366.1229420785
monthly sales:13883
picture url: https://cf.shopee.tw/file/0113811cd229ae8abefafc3a30f25b88

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:[現貨] 玩美日記面膜水絲布面膜(1片入/25ml) 多款供選 24H出貨 台灣現貨 【小麥購物】【S004】面膜
price info:$8
price_min:8
price_max:8
product url: https://shopee.tw/-現貨-玩美日記面膜水絲布面膜(1片入-25ml)-多款供選-24H出貨-台灣現貨-【小麥購物】【S004】面膜-i.9687994.1211636185
monthly sales:12052
picture url: https://cf.shopee.tw/file/1beafca3faef69b9d42f71773fb80bcf

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:OK萊爾富可寄★美保★Mi★極潤保濕美肌天絲面膜~EGF馬油.玻尿酸.雪肌妍.金箔 超導黃金.白金.黑面膜
price info:$10 - $15
price_min:10
price_max:15
product url: https://sh

category:美妝保健
class name:開架流行清潔保養
product name:正品現貨附發票【3124】特價ETUDE HOUSE 天天敷肌蜜面膜 玻尿酸面膜 玻尿酸 保濕面膜 etudehouse
price info:$19
price_min:19
price_max:19
product url: https://shopee.tw/正品現貨附發票【3124】特價ETUDE-HOUSE-天天敷肌蜜面膜-玻尿酸面膜-玻尿酸-保濕面膜-etudehouse-i.2543564.58848984
monthly sales:1764
picture url: https://cf.shopee.tw/file/a40423a0d9cc82ceba3f2eaead20b2f1

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:RANESS 胺基酸奇蹟洗顏乳❄️現貨供應免運
price info:$720
price_min:720
price_max:720
product url: https://shopee.tw/RANESS-胺基酸奇蹟洗顏乳❄️現貨供應免運-i.4091791.1272425916
monthly sales:1241
picture url: https://cf.shopee.tw/file/315bc1a016021ea864d986060dc7b8c0

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:imju薏仁清潤化妝水500ml【康是美】
price info:$225
price_min:225
price_max:225
product url: https://shopee.tw/imju薏仁清潤化妝水500ml【康是美】-i.33206500.606977178
monthly sales:1228
picture url: https://cf.shopee.tw/file/db2e40b786ca9cdabc982de73a9169a1

save to mongodb
category:美妝保健
class name:開架流行清潔保養
prod

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:【奈奈美代購誌】免運+現貨 The Ordinary 菸鹼胺 杜鵑花酸 五胜肽 果酸 藍銅胜肽 精華液 維他命c 美白
price info:$210 - $609
price_min:210
price_max:609
product url: https://shopee.tw/【奈奈美代購誌】免運-現貨-The-Ordinary-菸鹼胺-杜鵑花酸-五胜肽-果酸-藍銅胜肽-精華液-維他命c-美白-i.16448224.911851682
monthly sales:935
picture url: https://cf.shopee.tw/file/74684fcf339ccce3ece999bbc8cdec34

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:#現貨+預購《Abby德國代購》德國Dm Balea時空膠囊 ❤️全館超低價歡慶開幕🎉
price info:$47 - $49
price_min:47
price_max:49
product url: https://shopee.tw/-現貨-預購《Abby德國代購》德國Dm-Balea時空膠囊-❤️全館超低價歡慶開幕🎉-i.5723558.1837282854
monthly sales:1894
picture url: https://cf.shopee.tw/file/8bc4b5ea2bb2b8fe493b679cb40d5df1

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:【現貨】日本 熊野 麗白洗卸/薄荷涼感/白樁油洗卸/薏仁保濕/無添加 zero 洗面乳 洗卸兩用 潔面乳
price info:$59 - $69
price_min:59
price_max:69
product url: https://shopee.tw/【現貨】日本-熊野-麗白洗卸-薄荷涼感-白樁油洗卸-薏仁保濕-無添加-zero-洗面乳-洗卸兩用-潔面乳-i.4514571.347445267
monthly sales:

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:《開春優惠》ROHTO肌研 化妝水 極潤保濕化妝水/白潤美白化妝水 保濕乳液/美白乳液【日】170/140ml
price info:$249
price_min:249
price_max:249
product url: https://shopee.tw/《開春優惠》ROHTO肌研-化妝水-極潤保濕化妝水-白潤美白化妝水-保濕乳液-美白乳液【日】170-140ml-i.44584288.821489374
monthly sales:620
picture url: https://cf.shopee.tw/file/c53940595d2242f0ac791ad476a04d79

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:MEDIHEAL 美迪惠爾 - 高效特強x3倍保濕 亮白淡斑 茶樹舒缓 抗皺活膚 舒緩 乳清蛋白 面膜
price info:$35 - $45
price_min:35
price_max:45
product url: https://shopee.tw/MEDIHEAL-美迪惠爾-高效特強x3倍保濕-亮白淡斑-茶樹舒缓-抗皺活膚-舒緩-乳清蛋白-面膜-i.5354709.199607391
monthly sales:619
picture url: https://cf.shopee.tw/file/5ca24215ceb4859d135d8161f8b562af

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:台灣製 MIT 極水潤/蠟菊修護/三倍皙淨透亮/馬油潤澤 面膜(多款可選/單片)-MI
price info:$10
price_min:10
price_max:10
product url: https://shopee.tw/台灣製-MIT-極水潤-蠟菊修護-三倍皙淨透亮-馬油潤澤-面膜(多款可選-單片)-MI-i.933980.43411705
monthly sales:1797
picture url: https://cf.

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:【梨大小姐限時特價】Somebymi 30天奇蹟水 奇蹟皂 奇蹟精華液 化妝水 Some by mi
price info:$95 - $399
price_min:95
price_max:399
product url: https://shopee.tw/【梨大小姐限時特價】Somebymi-30天奇蹟水-奇蹟皂-奇蹟精華液-化妝水-Some-by-mi-i.32084592.1106956346
monthly sales:502
picture url: https://cf.shopee.tw/file/bdb8d165fd30fe41326b3c9057398ff5

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:【梨大小姐】AHC 最新第八代眼霜 七代眼霜 六代眼霜 全效多功能眼霜 眼霜 12ml A.H.C
price info:$64 - $229
price_min:64
price_max:229
product url: https://shopee.tw/【梨大小姐】AHC-最新第八代眼霜-七代眼霜-六代眼霜-全效多功能眼霜-眼霜-12ml-A.H.C-i.32084592.908304942
monthly sales:502
picture url: https://cf.shopee.tw/file/fe631cfcb29a158f7ee36539fc4e6ebf

save to mongodb
category:美妝保健
class name:開架流行清潔保養
product name:附發票💹🚀現貨🚀[新改款]卸睫眼膜/卸除眼膜/卸睫墊片/卸除墊片【PJOB smallc5168】
price info:$5
price_min:5
price_max:5
product url: https://shopee.tw/附發票💹🚀現貨🚀-新改款-卸睫眼膜-卸除眼膜-卸睫墊片-卸除墊片【PJOB-smallc5168】-i.10444370.91970319
monthly sales:1603
picture url: htt

...

picture load 50/50...
category:寵物
class name:其他
product name:{板橋新埔門市} 心僕鮮食【貓咪】生肉餐 100g
price info:$50 - $55
price_min:50
price_max:55
product url: https://shopee.tw/-板橋新埔門市-心僕鮮食【貓咪】生肉餐-100g-i.2628232.845499687
monthly sales:997
picture url: https://cf.shopee.tw/file/31bc26ada35cb6bff28fe3abfc0a2519

save to mongodb
category:寵物
class name:其他
product name:台灣發貨 塵不染除毛磚 除毛磚 FB同款 神奇除毛磚 除毛屑 毛小孩 除毛達人 除毛神器 韓國神奇除毛磚 一塵不染除毛磚
price info:$55
price_min:55
price_max:55
product url: https://shopee.tw/台灣發貨-塵不染除毛磚-除毛磚-FB同款-神奇除毛磚-除毛屑-毛小孩-除毛達人-除毛神器-韓國神奇除毛磚-一塵不染除毛磚-i.2162114.1777323828
monthly sales:654
picture url: https://cf.shopee.tw/file/d7b0a37a7274de422619a70bd897418a

save to mongodb
category:寵物
class name:其他
product name:附影片🌸DIY百變組合寵物籠🌸網片 魔片 網門圍欄 貓 兔子籠子 創意組合 鐵藝書架 網格掛牆 狗柵欄 貓 兔子籠子
price info:$3 - $50
price_min:3
price_max:50
product url: https://shopee.tw/附影片🌸DIY百變組合寵物籠🌸網片-魔片-網門圍欄-貓-兔子籠子-創意組合-鐵藝書架-網格掛牆-狗柵欄-貓-兔子籠子-i.23026031.1212849544
monthly sales:755
picture url: https://cf.shopee.tw/file/b4a9980c83

save to mongodb
category:女鞋
class name:休閒鞋、帆布鞋
product name:Vans板鞋 男女鞋 16FW Supreme Vans Authentic pro 棋盤格 黑白格 男鞋 女鞋 懶人鞋
price info:$440 - $640
price_min:440
price_max:640
product url: https://shopee.tw/Vans板鞋-男女鞋-16FW-Supreme-Vans-Authentic-pro-棋盤格-黑白格-男鞋-女鞋-懶人鞋-i.32397582.531815016
monthly sales:644
picture url: https://cf.shopee.tw/file/e1c7228d5f9e19e8030e20e367e5ce0e

save to mongodb
category:女鞋
class name:休閒鞋、帆布鞋
product name:限時狂歡價 Vans Old Skool 范斯 基本款 經典款 黑白 滑板鞋 男 女 情侶鞋 VN000D3HY28
price info:$390
price_min:390
price_max:390
product url: https://shopee.tw/限時狂歡價-Vans-Old-Skool-范斯-基本款-經典款-黑白-滑板鞋-男-女-情侶鞋-VN000D3HY28-i.87402472.1795014610
monthly sales:501
picture url: https://cf.shopee.tw/file/519c4b17662d7458a93aec860994bbe6

save to mongodb
class url: https://shopee.tw/鞋材用品-cat.64.17156?page=0&sortBy=sales , page: 0 ...
picture load 50/50...
category:女鞋
class name:鞋材用品
product name:【12H出貨！4D記憶棉 慢回彈不變形】增高鞋墊 增高墊 鞋底墊高 隱形鞋墊 內增高鞋墊 後跟墊 減壓透氣 久站 可裁剪
price info:$69 - $99
price_min:69
pri

save to mongodb
category:女生包包-精品
class name:皮夾
product name:現貨HelloKitty女生卡通旅行防水化妝包大容量卡通化妝包女收納包Kitty洗漱包化妝包收納零錢包化裝包美妝收納包
price info:$41
price_min:41
price_max:41
product url: https://shopee.tw/現貨HelloKitty女生卡通旅行防水化妝包大容量卡通化妝包女收納包Kitty洗漱包化妝包收納零錢包化裝包美妝收納包-i.26209487.1394881811
monthly sales:501
picture url: https://cf.shopee.tw/file/930cd10b5b47cdefb01466b8da7453c6

save to mongodb
category:女生包包-精品
class name:皮夾
product name:🔥現貨批發🔥 韓版糖果色多色小化妝包收納折疊化妝包包贈品水餃包新品熱賣
price info:$9 - $12
price_min:9
price_max:12
product url: https://shopee.tw/🔥現貨批發🔥-韓版糖果色多色小化妝包收納折疊化妝包包贈品水餃包新品熱賣-i.63479477.1229672916
monthly sales:560
picture url: https://cf.shopee.tw/file/99c8a54835fb3e0e31039607f0c162e9

save to mongodb
category:女生包包-精品
class name:皮夾
product name:挑戰現貨最低價499免運台灣公司現貨附發票【B006】新款大容量化妝包防水旅行包收納包梳洗包整理袋LBD
price info:$8 - $9
price_min:8
price_max:9
product url: https://shopee.tw/挑戰現貨最低價499免運台灣公司現貨附發票【B006】新款大容量化妝包防水旅行包收納包梳洗包整理袋LBD-i.61065527.1266768228
monthly sales:759
picture url: https://cf.shopee.tw/

picture load 50/50...
category:男生包包與配件
class name:飾品
product name:【64款】韓版大小款嘻哈經典款 鈦鋼耳夾 醫療鋼 夾式耳環 無耳洞 歐美耳環 耳環 耳針 男生 配件 圈圈耳環 圓圈耳環
price info:$20 - $35
price_min:20
price_max:35
product url: https://shopee.tw/【64款】韓版大小款嘻哈經典款-鈦鋼耳夾-醫療鋼-夾式耳環-無耳洞-歐美耳環-耳環-耳針-男生-配件-圈圈耳環-圓圈耳環-i.8619106.50758738
monthly sales:2281
picture url: https://cf.shopee.tw/file/1354d70fd062857a6905ba15d7586117

save to mongodb
category:男生包包與配件
class name:飾品
product name:🔥 300款【鈦鋼耳環】耳夾/磁鐵耳環/無耳洞耳環/夾式耳環/圓圈耳環/圈圈耳環【MG】
price info:$32 - $55
price_min:32
price_max:55
product url: https://shopee.tw/🔥-300款【鈦鋼耳環】耳夾-磁鐵耳環-無耳洞耳環-夾式耳環-圓圈耳環-圈圈耳環【MG】-i.4072023.322959388
monthly sales:751
picture url: https://cf.shopee.tw/file/9fd52d3020d72e94cb2a1beddfaefe15

save to mongodb
category:男生包包與配件
class name:飾品
product name:【300款】韓版鈦鋼耳環🍀LIZZWORLD🍀 無耳洞鋼耳夾/夾式耳環/圈圈耳環 圓圈耳環【G001】
price info:$32
price_min:32
price_max:32
product url: https://shopee.tw/【300款】韓版鈦鋼耳環🍀LIZZWORLD🍀-無耳洞鋼耳夾-夾式耳環-圈圈耳環-圓圈耳環【G001】-i.4072023.323304941
monthly sales:661
picture ur

save to mongodb
category:男生包包與配件
class name:襪子
product name:黑 白 灰都有 物超所值 MIT 台灣社頭 襪子 短襪 學生襪 中筒襪 1/2襪襪 男女襪 工作襪
price info:$10
price_min:10
price_max:10
product url: https://shopee.tw/黑-白-灰都有-物超所值-MIT-台灣社頭-襪子-短襪-學生襪-中筒襪-1-2襪襪-男女襪-工作襪-i.1730764.310362360
monthly sales:560
picture url: https://cf.shopee.tw/file/d3e41be65de7d47d48f77ce8c7315b92

save to mongodb
category:男生包包與配件
class name:襪子
product name:大促銷 黑襪 台灣製社頭襪 1/2 襪 中筒襪 學生襪 短襪 男襪 女襪 工作襪
price info:$10
price_min:10
price_max:10
product url: https://shopee.tw/大促銷-黑襪-台灣製社頭襪-1-2-襪-中筒襪-學生襪-短襪-男襪-女襪-工作襪-i.4753998.35591376
monthly sales:619
picture url: https://cf.shopee.tw/file/6b28188cb6c3cc4b642ff9964a9c007b

save to mongodb
class url: https://shopee.tw/眼鏡-cat.1837.1853?page=0&sortBy=sales , page: 0 ...
picture load 50/50...
class url: https://shopee.tw/手提包-cat.1837.1847?page=0&sortBy=sales , page: 0 ...
picture load 50/50...
class url: https://shopee.tw/圍巾、手套-cat.1837.1857?page=0&sortBy=sales , page: 0 ...
picture load 50/50...
class u

## 首页分类

In [10]:
from selenium import webdriver
import time,re,csv,pymongo,os
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys

def get_category(driver,url,save_file):
    driver.get(url) 
    time.sleep(2)
    close_popup(driver)
    for i in range(9):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
    html = driver.page_source
    doc = pq(html,parser="html")
    category_list = doc('.footer__category-list-item')
    #print(category_list)
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for i in category_list.items():
            category_info = i.find('.footer__category-list-item-main a')
            category = category_info.text()
            category_url = url + category_info.attr('href') 
            sub_list = i.find('.footer__category-list-item-sub')
            for sub in sub_list.items():
                main_class = sub.find('a').text() 
                class_url = url + sub.find('a').attr('href')    
                item = {
                            'category':category,
                            'category url':category_url,
                            'main class':main_class,
                            'class url':class_url,
                        }
                print('{},{},{},{}'.format(category,category_url,main_class,class_url))
                csvfile.write('{},{},{},{}\n'.format(category,category_url,main_class,class_url))
                save_to_mongo(item)

def close_popup(driver):
    driver.find_element_by_css_selector('.shopee-popup__close-btn').click()
                
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopee_crawl'
    MONGO_COLLECTION = 'shopee_tw_category'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


url = r'https://shopee.tw'
save_file = r'shopee_tw_category.csv'

if __name__ == '__main__':
    
    options = webdriver.ChromeOptions()
    prefs={
        'profile.default_content_setting_values':
        {
        'notifications':2
        }
    }
    options.add_experimental_option('prefs',prefs)
    driver = webdriver.Chrome(chrome_options = options)
    driver.maximize_window()
        
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['category','category url','main class','class url']) 
    
    get_category(driver,url,save_file)
    driver.close()

女生衣著,https://shopee.tw/女生衣著-cat.62,T恤,https://shopee.tw/T恤-cat.62.2164
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,小可愛/背心,https://shopee.tw/小可愛-背心-cat.62.2165
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,上衣類,https://shopee.tw/上衣類-cat.62.1473
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,帽T/大學T,https://shopee.tw/帽T-大學T-cat.62.2560
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,襯衫,https://shopee.tw/襯衫-cat.62.1475
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,針織/毛衣,https://shopee.tw/針織-毛衣-cat.62.2561
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,洋裝,https://shopee.tw/洋裝-cat.62.1485
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,短褲,https://shopee.tw/短褲-cat.62.1481
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,長褲,https://shopee.tw/長褲-cat.62.1479
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,裙子,https://shopee.tw/裙子-cat.62.1483
save to mongodb
女生衣著,https://shopee.tw/女生衣著-cat.62,外套,https://shopee.tw/外套-cat.62.1487
save to mongodb
女生衣著,https://shopee

save to mongodb
男鞋,https://shopee.tw/男鞋-cat.1859,鞋材用品,https://shopee.tw/鞋材用品-cat.1859.7889
save to mongodb
男鞋,https://shopee.tw/男鞋-cat.1859,其他,https://shopee.tw/其他-cat.1859.1860
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,外出用品,https://shopee.tw/外出用品-cat.1657.1663
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,寵物零食,https://shopee.tw/寵物零食-cat.1657.2698
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,貓飼料,https://shopee.tw/貓飼料-cat.1657.2672
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,狗飼料,https://shopee.tw/狗飼料-cat.1657.2671
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,其他寵物食品,https://shopee.tw/其他寵物食品-cat.1657.2673
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,寵物用品,https://shopee.tw/寵物用品-cat.1657.2205
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,寵物玩具,https://shopee.tw/寵物玩具-cat.1657.2674
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,寵物服飾,https://shopee.tw/寵物服飾-cat.1657.1661
save to mongodb
寵物,https://shopee.tw/寵物-cat.1657,水族相關,https://shopee.tw/水族相關-cat.1657.2675
sav

save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,手錶,https://shopee.tw/手錶-cat.2580.2589
save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,戒指,https://shopee.tw/戒指-cat.2580.2585
save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,吊飾,https://shopee.tw/吊飾-cat.2580.2591
save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,國際精品配件,https://shopee.tw/國際精品配件-cat.2580.18851
save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,皮帶/腰帶,https://shopee.tw/皮帶-腰帶-cat.2580.18849
save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,黃金,https://shopee.tw/黃金-cat.2580.18863
save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,鑽石,https://shopee.tw/鑽石-cat.2580.18861
save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,珠寶/玉石,https://shopee.tw/珠寶-玉石-cat.2580.18865
save to mongodb
女生配件,https://shopee.tw/女生配件-cat.2580,其他,https://shopee.tw/其他-cat.2580.2592
save to mongodb
戶外與運動用品,https://shopee.tw/戶外與運動用品-cat.1611,登山、露營,https://shopee.tw/登山、露營-cat.1611.2637
save to mongodb
戶外與運動用品,https://shopee.tw/戶外與運動用品-cat

In [16]:
from selenium import webdriver
import time,re,csv,pymongo,os
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys


def get_list(driver,save_file,category,class_name,class_url,url_format,price_select,sales_select,page,category_en):
    
    target_url = class_url.format(url_format,page)
    print('class url: {} , page: {} ...'.format(target_url,page))
    driver.get(target_url) 
    time.sleep(2)
    for i in range(5):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(4)
    pic_num = len(driver.find_elements_by_css_selector('._1T9dHf._3XaILN'))
    print('picture load {}/50...'.format(pic_num))
    while (pic_num<49):
        driver.refresh()
        for i in range(4):
            driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
        for i in range(4):
            driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_UP)
            time.sleep(2)
        pic_num = len(driver.find_elements_by_css_selector('._1T9dHf._3XaILN'))
        print('picture load again {}/50...'.format(pic_num))

    html = driver.page_source
    doc = pq(html,parser="html")
    item_list = doc('.col-xs-2-4.shopee-search-item-result__item')
    num = 0
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for i in item_list.items():
            try:
                sales_info = i.find('._2-i6yP').text()
                
                monthly_sales_info = re.search('月銷量 (\d+)',sales_info)
                if monthly_sales_info:
                    monthly_sales = int(monthly_sales_info.group(1))
                else:
                    monthly_sales = 0
                if monthly_sales > sales_select:
                    num += 1
                    price_info  = i.find('._1w9jLI._37ge-4._2XtIUk').text()
                    price_range = re.sub(r'\,|\n','',price_info)
                    #print(price_range)
                    if '-' in price_range:
                        a = re.findall('\$(\d+) - \$(\d+)',price_range)
                        price_min = int(a[0][0])
                        price_max = int(a[0][1])
                    else:
                        price = int(re.search('\$(\d+)',price_range).group(1))
                        price_min = price
                        price_max = price
                    if price_max > price_select:
                        pro_name = i.find('._1NoI8_._2gr36I').text()
                        pro_name = re.sub(r',',' ',pro_name)
                        sales_info = i.find('._2-i6yP').text()
                        #print(sales_info)
                        prefix = 'https://shopee.tw'
                        pro_url = prefix+i.find('div a').attr('href') 
                        pic_info = i.find('._1T9dHf._3XaILN').attr('style')
                        #print(type(pic_info))
                        pic_url = re.search(r'url\("(.*)_tn"\);',str(pic_info)).group(1).strip()
                        #print(pic_url)
                        print('category:{}\nclass name:{}\nproduct name:{}\nprice info:{}\nprice_min:{}\nprice_max:{}\nproduct url: {}\nmonthly sales:{}\npicture url: {}\n'.format(category,class_name,pro_name,price_range,price_min,price_max,pro_url,monthly_sales,pic_url))
                        csvfile.write('{},{},{},{},{},{},{},{},{}\n'.format(category,class_name,pro_name,price_range,price_min,price_max,monthly_sales,pro_url,pic_url)) 
                        pic_name = ' '.join(re.sub('[^a-z0-9A-Z_\-\s]','',pro_name,count=0).split())
                        #save_image(pic_url,pic_name,category)
                        item = {
                            'category':category,
                            'main class':class_name,
                            'item name':pro_name,
                            'price info':price_range,
                            'price min':price_min,
                            'price max':price_max,
                            'monthly sales':monthly_sales,
                            'item url':pro_url,
                            'item picture':pic_url
                        }
                        save_to_mongo(item)
            except Exception as e:
                print('something got worng {}, pass this item {}'.format(e,pro_name))
                continue       
    #print(num,len(item_list))
    if num == len(item_list):
        page += 1
        get_list(driver,save_file,category,class_name,class_url,url_format,price_select,sales_select,page)
        
def save_image(img_url,pic_name,category):
    print('{} saving picture... {}'.format(pic_name,img_url))
    img_response = requests.get(img_url,headers=headers)
    path = r'{}'.format(category)
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
    with open(r'{}\{}.png'.format(path,pic_name),'ab') as f:
        f.write(img_response.content)          

def save_to_mongo(result,category_en):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopee_tw'
    MONGO_COLLECTION = 'shopee_tw_{}'.format(category_en)
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


def get_url(file,driver,save_file,price_select,sales_select,page,category_select,category_en): 
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['category','main class','item name','price range','price_min','price_max','monthly sales','item url','item picture']) 
    with open(file,'r',encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = [row for row in reader]
        for row in rows[1:]:
            category = row[0]
            if category == category_select:
                class_name = row[2]
                url_format = row[3]
                get_list(driver,save_file,category,class_name,class_url,url_format,price_select,sales_select,page,category_en)
            
file = 'shopee_tw_category.csv'  
headers = {'User-Agent': 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}            
class_url = '{}?page={}&sortBy=sales'
page = 0
price_select = 0
sales_select = 500
category_select = '家電影音'
category_en = 'homeappliances'
save_file = r'shopee_tw_{}.csv'.format(category_en)


if __name__ == '__main__':
    driver = webdriver.Chrome()
    driver.maximize_window()    
    get_url(file,driver,save_file,price_select,sales_select,page,category_select,category_en)
    driver.close()

class url: https://shopee.tw/生活家電-cat.71.2627?page=0&sortBy=sales , page: 0 ...
picture load 50/50...
category:家電影音
class name:生活家電
product name:【現貨特價】小米靜電過濾棉 靜電過濾棉 空氣清淨機 空調濾網 靜電 PM2.5 防塵 小米静電棉 小米淨化器 小米pro
price info:$16
price_min:16
price_max:16
product url: https://shopee.tw/【現貨特價】小米靜電過濾棉-靜電過濾棉-空氣清淨機-空調濾網-靜電-PM2.5-防塵-小米静電棉-小米淨化器-小米pro-i.4093582.832823887
monthly sales:2343
picture url: https://cf.shopee.tw/file/1fcac3fe0b511728b5e864c32b504806

something got worng save_to_mongo() missing 1 required positional argument: 'category_en', pass this item 【現貨特價】小米靜電過濾棉 靜電過濾棉 空氣清淨機 空調濾網 靜電 PM2.5 防塵 小米静電棉 小米淨化器 小米pro
category:家電影音
class name:生活家電
product name:日本 Maxell 新版公司貨 3V 鈕扣電池 CR2032 CR2016 CR2025 寶可夢手環 水銀電池 鈕扣
price info:$6 - $10
price_min:6
price_max:10
product url: https://shopee.tw/日本-Maxell-新版公司貨-3V-鈕扣電池-CR2032-CR2016-CR2025-寶可夢手環-水銀電池-鈕扣-i.14187170.909317228
monthly sales:4051
picture url: https://cf.shopee.tw/file/40016e3b1dcd00cb9df6b0fbc4a02754

something got worng save_to_mong

picture load 50/50...
class url: https://shopee.tw/美容-健康家電-cat.71.2628?page=0&sortBy=sales , page: 0 ...
picture load 50/50...
category:家電影音
class name:美容/健康家電
product name:【大量現貨】現貨大特價【24Ｈ快速出貨】【湘湘小舖*繁體版體重計】體重計 體重機 情人節禮物 瘦身 減肥 減重
price info:$5 - $329
price_min:5
price_max:329
product url: https://shopee.tw/【大量現貨】現貨大特價【24Ｈ快速出貨】【湘湘小舖*繁體版體重計】體重計-體重機-情人節禮物-瘦身-減肥-減重-i.19782903.610840975
monthly sales:7305
picture url: https://cf.shopee.tw/file/31b3d6426e12ee9841b0175b96e06baf

something got worng save_to_mongo() missing 1 required positional argument: 'category_en', pass this item 【大量現貨】現貨大特價【24Ｈ快速出貨】【湘湘小舖*繁體版體重計】體重計 體重機 情人節禮物 瘦身 減肥 減重
category:家電影音
class name:美容/健康家電
product name:【SS】當天出貨 納米 噴霧器 補水儀 20ml水箱 納米補水儀 便攜 保濕噴霧 補水神器 保濕神器 加濕器 手持噴霧器
price info:$76
price_min:76
price_max:76
product url: https://shopee.tw/【SS】當天出貨-納米-噴霧器-補水儀-20ml水箱-納米補水儀-便攜-保濕噴霧-補水神器-保濕神器-加濕器-手持噴霧器-i.75625367.1269886881
monthly sales:1485
picture url: https://cf.shopee.tw/file/5176800efe824fb131aff96d03f168af

somethin

picture load 50/50...
category:家電影音
class name:音響、喇叭
product name:藍芽喇叭、行動電源、娃娃機商品、娃娃機批發、美好2025、2088、2055、007、史迪奇喇叭、kitty喇叭、品韻
price info:$350 - $850
price_min:350
price_max:850
product url: https://shopee.tw/藍芽喇叭、行動電源、娃娃機商品、娃娃機批發、美好2025、2088、2055、007、史迪奇喇叭、kitty喇叭、品韻-i.1710806.1471491560
monthly sales:592
picture url: https://cf.shopee.tw/file/ac84d1b3d0e727ecfd3ba03208a60c02

something got worng save_to_mongo() missing 1 required positional argument: 'category_en', pass this item 藍芽喇叭、行動電源、娃娃機商品、娃娃機批發、美好2025、2088、2055、007、史迪奇喇叭、kitty喇叭、品韻
class url: https://shopee.tw/大型家電-cat.71.11894?page=0&sortBy=sales , page: 0 ...
picture load 50/50...
category:家電影音
class name:大型家電
product name:洗衣機濾網 棉絮過濾網 東元.國際.聲寶.三洋.夏普.東芝.日立.歌林量大可議價
price info:$30 - $35
price_min:30
price_max:35
product url: https://shopee.tw/洗衣機濾網-棉絮過濾網-東元.國際.聲寶.三洋.夏普.東芝.日立.歌林量大可議價-i.16864936.314739991
monthly sales:945
picture url: https://cf.shopee.tw/file/4228b7daa8a693c41b18bc5c1e918e66

something got worng save_to_mongo() 

In [1]:
from urllib.request import quote, unquote

url = 'https://shopee.tw/女生配件-cat.2580'
test = quote(url,safe=";/?:@&=+$,", encoding="utf-8")
print(test)


https://shopee.tw/%E5%A5%B3%E7%94%9F%E9%85%8D%E4%BB%B6-cat.2580
